<a href="https://colab.research.google.com/github/kmalhotra18/Airline-Chatbot/blob/main/Airline_Chatbot_with_Spanish_Multi_Modal_Image_Generation_and_Audio_Claude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install anthropic

In [ ]:
!pip install gradio

In [ ]:
# Imports
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
import anthropic
import json
import base64
from io import BytesIO
from PIL import Image
from pydub import AudioSegment
import gradio as gr                           # Import Gradio for UI
from IPython.display import Audio, display

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
claude = anthropic.Anthropic(api_key=anthropic_api_key)

In [ ]:
# System behavior
system_message = (
    "You are a helpful assistant for an airline called FlightAI. "
    "Give short, courteous answers, no more than 1 sentence. "
    "Always be accurate. If you don't know the answer, say so."
)

In [ ]:
# Mock ticket prices
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

In [ ]:
def get_ticket_price(destination_city):
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")


In [ ]:
# Detect city from user message
def detect_city_from_text(text):
    text = text.lower()
    for city in ticket_prices.keys():
        if city in text:
            return city
    return None

In [ ]:
# DALL-E image generator (still uses OpenAI)
from openai import OpenAI
openai = OpenAI()
def artist(city):
    try:
        print(f"Generating image for {city}...")
        response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
        image_base64 = response.data[0].b64_json
        if not image_base64:
            print("No base64 image returned.")
            return None

        image_data = base64.b64decode(image_base64)
        image = Image.open(BytesIO(image_data))
        image.save("generated_tokyo.png")  # Save to check output
        print("Image saved to generated_tokyo.png")
        return image
    except Exception as e:
        print(f"Image generation failed: {e}")
        return None

In [ ]:
# Text-to-speech with OpenAI
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=message
    )
    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())
    return output_filename

In [ ]:
# Convert chat history for Claude
def convert_history_to_prompt(history):
    prompt = ""
    for turn in history:
        if turn["role"] == "user":
            prompt += f"\nHuman: {turn['content']}"
        else:
            prompt += f"\nAssistant: {turn['content']}"
    prompt += "\nAssistant:"
    return prompt.strip()

In [ ]:
def translate_to_spanish_claude(text):
    try:
        response = claude.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=400,
            temperature=0.3,
            system="You are a professional translator. Translate the following English text to natural, fluent Spanish. Do not explain. Only give the translation.",
            messages=[
                {"role": "user", "content": text}
            ]
        )
        return response.content[0].text.strip()
    except Exception as e:
        return f"(Error in translation: {e})"

In [ ]:
def chat_claude(history):
    messages = [{"role": "user", "content": convert_history_to_prompt(history)}]
    response = claude.messages.create(
        model="claude-3-haiku-20240307",
        system=system_message,
        max_tokens=1000,
        temperature=0.7,
        messages=messages
    )

    reply = response.content[0].text.strip()
    image = None
    audio_path = None

    # Translate to Spanish using Claude again
    spanish_translation = translate_to_spanish_claude(reply)

    # Simulated tool call: city-based response and image
    city = detect_city_from_text(reply)
    if city:
        price = get_ticket_price(city)
        reply = f"The price to {city.title()} is {price}."
        spanish_translation = translate_to_spanish_claude(reply)
        image = artist(city)

    audio_path = talker(reply)
    history.append({"role": "assistant", "content": reply})

    return history, image, audio_path, spanish_translation

In [ ]:
# Gradio UI with option to display Spanish responses:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
        audio_output = gr.Audio(label="Voice", autoplay=True)
        spanish_output = gr.Markdown(label="Traducción al Español")
    with gr.Row():
        entry = gr.Textbox(label="Chat with FlightAI:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role": "user", "content": message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
    chat_claude, inputs=chatbot, outputs=[chatbot, image_output, audio_output, spanish_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)